In [38]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression

In [15]:
df = pd.read_csv('../../datasets/processed_data/combined_features/ELGIEQUIP.BO.csv')

In [16]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'low', 'high',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'inr=x_percent_change', 'cl=f_percent_change',
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change'],
      dtype='object', length=635)

In [17]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [18]:
df = create_custom_target(df)

In [19]:
df['yesterday_close']

1        13.562500
2        13.225000
3        13.175000
4        12.812500
5        12.787500
           ...    
3745    571.549988
3746    571.549988
3747    556.349976
3748    543.400024
3749    534.200012
Name: yesterday_close, Length: 3749, dtype: float64

In [20]:
df['ln_target'] 

1       0.021879
2       0.016870
3       0.022515
4       0.014528
5      -0.004900
          ...   
3745    0.019835
3746    0.020350
3747    0.011971
3748    0.012073
3749    0.027328
Name: ln_target, Length: 3749, dtype: float64

In [21]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [22]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [23]:
df_without_date = df.drop(columns=["date"])

In [24]:
df_without_date.dropna(inplace=True)

In [25]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [26]:
window_size = 10

In [27]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# further split test set to have an hold out set to be used for backtesting
dev_data, test_data = train_test_split(test_data, train_size=0.5, shuffle=False)

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(dev_data, window_size, 'ln_target')  

In [28]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [29]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [30]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling         window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [35]:
def evaluate_model(model, window, dev_data, dev_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(dev_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(dev_data, window, y_pred)

    dev_dates = dev_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': dev_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [61]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, dev_data, dev_date, X_test_50, y_test)

In [62]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-08-25  114.199997  118.901756
 1    2020-08-26  113.900002  116.738515
 2    2020-08-27  115.000000  117.821525
 3    2020-08-28  115.675003  118.694978
 4    2020-08-31  115.000000  118.403301
 ..          ...         ...         ...
 360  2021-12-30  304.000000  310.457884
 361  2021-12-31  302.600006  311.885857
 362  2022-01-03  314.350006  313.631705
 363  2022-01-04  314.950012  318.536497
 364  2022-01-05  319.700012  322.676155
 
 [365 rows x 3 columns],
 0.025224353827443452,
 6.250358269350869)

In [41]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 2.61%


In [42]:
feature_names = df.columns.tolist()

# Get feature importances
importances = model_50.feature_importances_

# Map importances to corresponding feature names
feature_importance_mapping = dict(zip(feature_names, importances))

# Sort by importance
sorted_features = sorted(feature_importance_mapping.items(), key=lambda x: x[1], reverse=True)

# Display top 10
for feature, importance in sorted_features[:10]:
    print(f"{feature}: {importance}")


trend_vortex_ind_diff: 0.04353187199639656
volatility_kcc: 0.04049639880235634
trend_sma_fast: 0.03839040422926578
trend_trix: 0.03703340360960847
trend_sma_slow: 0.03412387755592831
volume_em: 0.03407882098984366
volatility_kch: 0.03349736644603003
volume_vwap: 0.03320023705196796
trend_vortex_ind_neg: 0.0329914684554026
volatility_kchi: 0.03146572706858395


In [47]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

Repeat the process for the top 50 features without sentiment of ELGIEQUIP.BO

In [65]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [66]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [67]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [68]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [69]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [70]:
df_without_date.dropna(inplace=True)

In [71]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# further split test set to have an hold out set to be used for backtesting
dev_data, test_data = train_test_split(test_data, train_size=0.5, shuffle=False)

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(dev_data, window_size, 'ln_target')  

In [72]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [73]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [74]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, dev_data, dev_date, X_test_50, y_test)

In [75]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-08-25  114.199997  118.635986
 1    2020-08-26  113.900002  116.619891
 2    2020-08-27  115.000000  117.524057
 3    2020-08-28  115.675003  118.260606
 4    2020-08-31  115.000000  118.167544
 ..          ...         ...         ...
 360  2021-12-30  304.000000  311.141312
 361  2021-12-31  302.600006  313.651776
 362  2022-01-03  314.350006  314.137878
 363  2022-01-04  314.950012  319.145061
 364  2022-01-05  319.700012  324.344138
 
 [365 rows x 3 columns],
 0.02428908505027105,
 6.185062658208494)

In [59]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 2.52%


In [60]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)